In [2]:
import numpy as np

In [44]:
def state_to_ind(state):
    if state == 'AA':
        return 0
    if state == 'AG':
        return 1
    if state == 'GA':
        return 2
    if state == 'GG':
        return 3

def fb(Π, A, B, open_states):
    states_num = len(Π)
    n, m = open_states.shape
    
    α = np.zeros((states_num, n, m))
    
    for j in range(states_num):
        α[j, 0, 0] = Π[j] * B[j, state_to_ind(open_states[0, 0])]
        
    for t in range(1, n):
        for j in range(states_num):
            α[j, t, 0] = α[1, t - 1, 0] * A[1, j] * B[j, state_to_ind(open_states[t, 0])]

#         for j in range(states_num):
#             α[j, t, 0] = 0
#             for i in range(states_num):
#                 α[j, t, 0] += α[i, t - 1, 0] * A[i, j] * B[j, state_to_ind(open_states[t, 0])]

    for k in range(1, m):
        for j in range(states_num):
            α[j, 0, k] = α[2, 0, k - 1] * A[2, j] * B[j, state_to_ind(open_states[0, k])]
        
#         for j in range(states_num):
#             α[j, 0, k] = 0
#             for i in range(states_num):
#                 α[j, 0, k] += α[i, 0, k - 1] * A[i, j] * B[j, state_to_ind(open_states[0, k])]
                
#     for j in range(states_num):
#         print(α[j])
#         print()
    
    for t in range(1, n):
        for k in range(1, m):
            for j in range(states_num):
                α[j, t, k] = 0
                α[j, t, k] += α[0, t - 1, k - 1] * A[0, j] * B[j, state_to_ind(open_states[t, k])]
                α[j, t, k] += α[1, t - 1, k] * A[1, j] * B[j, state_to_ind(open_states[t, k])]
                α[j, t, k] += α[2, t, k - 1] * A[2, j] * B[j, state_to_ind(open_states[t, k])]

    β = np.zeros((states_num, n, m))
    β[:, -1, -1] = 1
    
    for t in range(n - 2, -1, -1):
        for i in range(states_num):
            β[i, t, -1] = β[1, t + 1, -1] * A[i, 1] * B[1, state_to_ind(open_states[t + 1, -1])]
        
#         for i in range(states_num):
#             β[j, t, -1] = 0
#             for j in range(states_num):
#                 β[j, t, -1] += β[j, t + 1, -1] * A[i, j] * B[j, state_to_ind(open_states[t + 1, -1])]

    for k in range(m - 2, -1, -1):
        for i in range(states_num):
            β[i, -1, k] += β[2, -1, k + 1] * A[i, 2] * B[2, state_to_ind(open_states[-1, k + 1])]
        
#         for i in range(states_num):
#             β[j, -1, k] = 0
#             for j in range(states_num):
#                 β[j, -1, k] += β[j, -1, k + 1] * A[i, j] * B[j, state_to_ind(open_states[-1, k + 1])]
    
    for t in range(n - 2, -1, -1):
        for k in range(m - 2, -1, -1):
            for i in range(states_num):
                β[i, t, k] = 0
                β[i, t, k] += β[0, t + 1, k + 1] * A[i, 0] * B[0, state_to_ind(open_states[t + 1, k + 1])]
                β[i, t, k] += β[1, t + 1, k] * A[i, 1] * B[1, state_to_ind(open_states[t + 1, k])]
                β[i, t, k] += β[2, t, k + 1] * A[i, 2] * B[2, state_to_ind(open_states[t, k + 1])]

    result = α * β / np.sum(α[:, -1, -1])
    
#     print()
#     for j in range(states_num):
#         print(result[j])
        
    return result[0]

In [53]:
δ = 0.1
ρ = 0.05
ϵ = 0.15

#                 M      X  Y
Π = np.array([1 - 2 * δ, δ, δ])

A = np.array([
    #  M        X  Y
    [1 - 2 * δ, δ, δ], # M
    [1 - ϵ - ρ, ϵ, ρ], # X
    [1 - ϵ - ρ, ρ, ϵ]  # Y
])

B = np.array([
    # AA  AG   GA   GG
    [0.49, 0.01, 0.01, 0.49], # M
    [0.25, 0.25, 0.25, 0.25], # X
    [0.25, 0.25, 0.25, 0.25] # Y
])

In [54]:
open_states = lambda s1, s2: np.array([[ch1 + ch2 for ch2 in s2] for ch1 in s1])

In [55]:
(fb(Π, A, B, open_states('AAAGAGAGGG', 'GAGAGG')) * 100).astype(int)

array([[ 15,   0,   0,   0,   0,   0],
       [  7,   7,   0,   0,   0,   0],
       [  1,  13,   0,   0,   0,   0],
       [106,   0,  16,   0,   0,   0],
       [  0, 111,   0,  11,   0,   0],
       [  0,   0, 117,   0,   5,   0],
       [  0,   0,   0, 121,   0,   0],
       [  0,   0,   0,   0,  84,   1],
       [  0,   0,   0,   0,  44,  43],
       [  0,   0,   0,   0,   0,  88]])